In [1]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


/home/brilworks/Desktop/Learning/model_finetuning/vevnv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


GPU Available: False
GPU Name: No GPU


In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

try:
    # Login to Hugging Face
    login(token="your_huggingface_token_here")  # Replace with your Hugging Face token
    # Check if GPU is available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    model_id = "meta-llama/Llama-3.2-3B-Instruct"

    # Load model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    ).to(device)

    tokenizer = AutoTokenizer.from_pretrained(model_id)

   

    # Create pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        device=device,
    )

   

except Exception as e:
    print(f"An error occurred: {str(e)}")

Using device: cpu


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 11.80it/s]
Device set to use cpu


In [14]:
 # Add padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

# Use the chat template for proper formatting
prompt = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id
)

print(outputs[0]["generated_text"])

RuntimeError: Numpy is not available

In [ ]:
import numpy
print(numpy.__version__)


2.3.0


In [26]:
from ollama import chat

stream = chat(
    model='llama3.2:1b',
    messages=[{'role': 'user', 'content': 'how are you?'}],
    stream=True
)

for chunk in stream:
    print(chunk['message']['content'], end='', flush=True)


I'm doing well, thank you for asking. Is there something I can help you with or would you like to talk about yourself?

In [19]:
!ollama create hasmukh_mer-llama3.2:1b -f Modelfile



Error: (line 4): command must be one of "from", "license", "template", "system", "adapter", "parameter", or "message"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from ollama import chat

response = chat(model='hasmukh-mer-llama3.2:1b', messages=[
    {'role': 'user', 'content': 'Who is Hasmukh Mer?'}
])

print(response['message']['content'])


I don't have any information on a person named Hasmukh Mer. Could you please provide more context or clarify who Hasmukh Mer is? I'll do my best to help.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./finetuned_cpu")
tokenizer = AutoTokenizer.from_pretrained("./finetuned_cpu")

prompt = "Who is hasmukh mer"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Map: 100%|██████████| 45/45 [00:00<00:00, 2161.86 examples/s]


ValueError: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`